In [2]:
from my_weapon import *
from SQLite_handler import *
from tqdm.notebook import tqdm as tqdm
from collections import defaultdict
plt.style.use('fivethirtyeight')

USA_STATES = ['NY', 'DC', 'IN', 'AR', 'WY', 'ME', 'TX', 'NH', 'CO', 'CA', 'IL',
             'WA', 'VA', 'FL', 'MA', 'OR', 'AZ', 'MT', 'MN', 'NE', 'TN', 'OH',
             'NJ', 'NV', 'KY', 'UT', 'NC', 'SC', 'PA', 'NM', 'KS', 'GA', 'MI',
             'WI', 'AK', 'MS', 'MD', 'LA', 'HI', 'MO', 'AL', 'CT', 'OK', 'IA',
             'WV', 'RI', 'SD', 'VT', 'ND', 'ID', 'DE']

# Rural and Urban

In [39]:
rural_table = pd.read_csv("data/ruralurbancodes2013.csv")
rural_table["Population"] = rural_table["Population_2010"].apply(lambda x: int(x.replace(",", "")))
rural_table["is_rural"] = rural_table["RUCC_2013"].apply(lambda x: 1 if x >= 3 else 0) # 分类，大于3以上是大城市
# rural_table[rural_table.County_Name=="Orleans Parish"]

# def county_to_rural(state_name, county_name):
#     try:
#         return rural_table[(rural_table["State"]==state_name) & (rural_table["county"]==county_name)]["is_rural"].iloc[0]
#     except:
#         return f"cannot find the loc ({state_name}, {county_name})"

dict_state_county_is_rural = defaultdict(dict)

for row in rural_table.itertuples():
    State = getattr(row, "State")
    county = getattr(row, "County_Name")
    is_rural = getattr(row, "is_rural")
    if county != "District of Columbia":
        county = " ".join(county.split()[: -1])
    dict_state_county_is_rural[State][county] = is_rural
    
# print(dict_state_county_is_rural["LA"])

rural_population = rural_table.groupby(["State", "is_rural"]).sum()["Population"]
rural_count = dict(rural_population)

In [40]:
rural_table

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description,Population,is_rural
0,1001,AL,Autauga County,"54,571",2,"Metro - Counties in metro areas of 250,000 to ...",54571,0
1,1003,AL,Baldwin County,"182,265",3,Metro - Counties in metro areas of fewer than ...,182265,1
2,1005,AL,Barbour County,"27,457",6,"Nonmetro - Urban population of 2,500 to 19,999...",27457,1
3,1007,AL,Bibb County,"22,915",1,Metro - Counties in metro areas of 1 million p...,22915,0
4,1009,AL,Blount County,"57,322",1,Metro - Counties in metro areas of 1 million p...,57322,0
...,...,...,...,...,...,...,...,...
3216,72145,PR,Vega Baja Municipio,"59,662",1,Metro - Counties in metro areas of 1 million p...,59662,0
3217,72147,PR,Vieques Municipio,"9,301",7,"Nonmetro - Urban population of 2,500 to 19,999...",9301,1
3218,72149,PR,Villalba Municipio,"26,073",2,"Metro - Counties in metro areas of 250,000 to ...",26073,0
3219,72151,PR,Yabucoa Municipio,"37,941",1,Metro - Counties in metro areas of 1 million p...,37941,0


## Load the users' opinions and locations

1. 累计预测模型结果（并没有细化到州）
2. 识别到地理位置的用户的观点（也可以看看全局结果）
3. 根据用户地理位置的经济状况，rescale后的结果

In [5]:
# 载入用户观点

# load users' opinion
# users_opinion = pd.read_csv("data/tweetid_userid_pro_hillary.csv")
import pendulum
users_opinion = defaultdict(lambda: [0, 0])
# set_users = set(users_loc.index)
# start = pendulum.datetime(2016, 9, 1)

# election date: 2016-11-8
# for line in tqdm(open("data/tweetid_userid_pro_hillary.csv")):
#     w = line.strip().split(",")
#     uid = int(w[0])
#     if uid in set_users:
#         if float(w[2]) < 0.5:
#             users_opinion[uid][0] += 1
#         else:
#             users_opinion[uid][1] += 1

users_opinion = []
i = 0
for line in tqdm(open("data/cul01-2020-07-01.csv")):
    w = line.strip().split(",")
    i += 1
    if i == 1:
        continue
    uid = str(w[0])
    users_opinion.append({
        "uid": uid,
        "Camp": "T" if int(w[2]) > int(w[1]) else "B" 
    })
users_opinion = pd.DataFrame(users_opinion).set_index("uid")
users_opinion

,Camp
uid,
314551800,T
1080662158415822849,B
3407179563,B
823998756655861760,B
79236840,T
...,...
14990801,T
4846877070,T
3182944844,T


In [6]:
users_opinion["Camp"].value_counts(normalize=True)

B    0.517661
T    0.482339
Name: Camp, dtype: float64

In [7]:
# 载入所有有地理位置信息的用户
users_loc = pd.read_csv("data/202006-all-users.csv", usecols=["uid", "state", "county"],
                        dtype={"uid": str, "state": str, "county": str},
                        index_col="uid")
users_loc = users_loc.dropna()
users_loc["county"] = users_loc["county"].apply(lambda x: " ".join(x.split(", ")[:-1]))
users_loc

,county,state
uid,,
140299630,Berkeley,CA
984471367176241152,Washington,DC
6497512,Hoboken,NJ
2808545932,Detroit,MI
357494668,Middletown,CT
...,...,...
1077379415867314176,Cedar Hill,TX
1084938692265820160,New York,NY
264400122,New York,NY


3072442个用户拥有state和county信息。一定要有state数据，county可能不存在。

这些人的观点呢？

In [8]:
users_opinion_with_location = users_loc.join(users_opinion)

In [9]:
users_opinion_with_location

,county,state,Camp
uid,,,
1000000328765493248,Hell,MI,T
100000075,San Francisco,CA,T
1000001371,New York,NY,B
1000001731,Los Angeles,CA,B
1000002740557373441,Mount Pleasant,SC,T
...,...,...,...
uid,,state,NaN
uid,,state,NaN
uid,,state,NaN


In [12]:
users_opinion_with_location["Camp"].value_counts()

B    1913099
T    1133785
Name: Camp, dtype: int64

In [13]:
users_opinion_with_location["Camp"].value_counts(normalize=True)

B    0.627887
T    0.372113
Name: Camp, dtype: float64

In [14]:
# Since we only know the name of city, but the county. We need convert the city to county.
# check some mapping from http://www.statsamerica.org/CityCountyFinder/Default.aspx

# 需要更加详细的转换
_dict_city_to_county = {
    'Chicago, IL': 'Cook',
    'Washington, DC': 'District of Columbia',
    'Atlanta, GA': 'Fulton',
    'Boston, MA': 'Suffolk',
    'Seattle, WA': 'King',
    'Brooklyn, NY': 'New York',
    'Miami, FL': 'Miami-Dade',
    'Las Vegas, NV': 'Clark',
    'Portland, OR': 'Multnomah',
    'Pittsburgh, PA': 'Allegheny',
    'San Antonio, TX': 'Bexar',
    'Nashville, TN': 'Davidson',
    'Orlando, FL': 'Orange',
    'Phoenix, AZ': 'Maricopa',
    'Indianapolis, IN': 'Marion',
    'Columbus, OH': 'Franklin',
    'Minneapolis, MN': 'Hennepin',
    'St Louis, MO': 'St. Louis',
    'Charlotte, NC': 'Mecklenburg',
    'Cleveland, OH': 'Cuyahoga',
    'Detroit, MI': 'Wayne',
    'Tampa, FL': 'Hillsborough',
    'New Orleans, LA': 'Orleans',
    'Kansas City, MO': 'Jackson',
    'Cincinnati, OH': 'Hamilton'
}
    
def city_to_county(state_name, city_name):
    return _dict_city_to_county.get(f"{city_name}, {state_name}", city_name)

In [15]:
from collections import Counter

find_county = Counter()
cannot_find_county = Counter()

for _state in tqdm(USA_STATES):
    _users = users_loc[users_loc.state==_state].copy()
    _users["county"] = _users["county"].apply(lambda x: city_to_county(_state, x))
    _users["is_rural"] = _users["county"].apply(lambda x: dict_state_county_is_rural[_state].get(x, -1))
    _users.to_csv(f"data/is_rural/{_state}.csv")

#     for row in _users.itertuples():
#         county = getattr(row, "county")
#         is_rural = getattr(row, "is_rural")
#         if is_rural == -1:
#             cannot_find_county[f"{county}, {_state}"] += 1
#         else:
#             find_county[f"{county}, {_state}"] += 1
    
    
# -1 represents that we cannot map the city (from BingMap) to county
# So we need more infomation between city and county

In [16]:
print(len(cannot_find_county))
print(cannot_find_county.most_common(10))
find_county.most_common(10)

0
[]


[]

从1月~6月，共12988437个用户

In [17]:
for _state in tqdm(USA_STATES):
    users_county = pd.read_csv("data/is_rural/" + _state + ".csv", index_col="uid")
    users_county = users_county.join(users_opinion)
    groups = dict(users_county.groupby(["is_rural", "Camp"]).size())
    # print(_state, groups) 

    w1 = rural_count.get((_state, 1), 0) / (rural_count.get((_state, 1), 0) + rural_count.get((_state, 0), 0))
    w0 = 1 - w1
    
    groups_T_H = dict(users_county[users_county.is_rural != -1].groupby("Camp").size())
    # print(groups_T_H)
    proT = groups_T_H.get("T", 0) / (groups_T_H.get("T", 0) + groups_T_H.get("B", 0))
    
    if (0, 'T') not in groups: # No urban people support Trump!
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1
    elif (1, 'T') not in groups: # No rural people support Trump!
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        rst = pro_T_0
    else:
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1 * w1 + pro_T_0 * w0
        
    print(f"{_state}, Census (rural areas): {w1 * 100:.2f}%, Trump: {proT * 100:.2f}%, After rescaling: {rst * 100:.2f}%")

KeyboardInterrupt: 

In [ ]:
users_counties = users_loc.join(users_opinion)

In [ ]:
users_counties

## 更细致的county转换

In [18]:
CityCounty = pd.read_csv("data/Borror_CityCounty.csv")

In [19]:
CityCounty

,placename,countyname,statefips,totpop,placefips,countyfips,pctpop
0,Aaronsburg CDP,Centre County (part),42,613,100,27,100.0
1,Aaronsburg CDP,Washington County (part),42,259,104,125,100.0
2,Abanda CDP,Chambers County (part),1,192,100,17,100.0
3,Abbeville city,Henry County (part),1,2688,124,67,100.0
4,Abbeville city,Abbeville County (part),45,5241,100,1,100.0
...,...,...,...,...,...,...,...
30849,Zuni Pueblo CDP,McKinley County (part),35,6302,86595,31,100.0
30850,Zurich city,Rooks County (part),20,99,81025,163,100.0
30851,Zwingle city,Dubuque County (part),19,65,87690,61,71.4
30852,Zwingle city,Jackson County (part),19,26,87690,97,28.6


In [20]:
CityCounty.columns

Index(['placename', 'countyname', 'statefips', 'totpop', 'placefips',
       'countyfips', 'pctpop'],
      dtype='object')

## 获取全国或是州的三变量比例

In [60]:
from pathlib import Path

path = Path("data/est2019-age-sex-county")

data = pd.DataFrame()

for in_name in path.rglob("*.csv"):
    # print(in_name)
    d = pd.read_csv(in_name, usecols=["STNAME", "CTYNAME", "YEAR", "AGE65PLUS_FEM", "AGE65PLUS_MALE", "AGE2024_MALE", "AGE2024_FEM", "AGE2529_MALE", "AGE2529_FEM", "AGE3034_MALE", "AGE3034_FEM", "AGE3539_MALE", "AGE3539_FEM", "AGE4044_MALE", "AGE4044_FEM", "AGE4549_MALE", "AGE4549_FEM", "AGE5054_MALE", "AGE5054_FEM", "AGE5559_MALE", "AGE5559_FEM", "AGE6064_MALE", "AGE6064_FEM"])
    # print(d)
    data = data.append(d)

data["County_Name"] = data["CTYNAME"]
data = data[data.YEAR == 12]
data

,STNAME,CTYNAME,YEAR,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE2024_MALE,AGE2024_FEM,AGE2529_MALE,AGE2529_FEM,AGE3034_MALE,...,AGE4044_FEM,AGE4549_MALE,AGE4549_FEM,AGE5054_MALE,AGE5054_FEM,AGE5559_MALE,AGE5559_FEM,AGE6064_MALE,AGE6064_FEM,County_Name
11,Georgia,Appling County,12,1478,1779,611,491,620,507,491,...,577,537,574,578,550,624,617,585,665,Appling County
23,Georgia,Atkinson County,12,573,610,272,273,280,267,269,...,257,265,247,297,237,241,264,240,265,Atkinson County
35,Georgia,Bacon County,12,815,1063,357,321,348,336,346,...,391,372,345,350,356,313,316,305,368,Bacon County
47,Georgia,Baker County,12,338,381,78,89,93,76,80,...,65,98,98,90,100,120,131,128,118,Baker County
59,Georgia,Baldwin County,12,3441,3957,2766,2664,1517,1329,1286,...,1033,1320,1138,1397,1234,1490,1435,1455,1408,Baldwin County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,Louisiana,Webster Parish,12,3334,4426,1123,1034,1254,1165,1131,...,1072,1099,1141,1201,1198,1293,1413,1264,1331,Webster Parish
731,Louisiana,West Baton Rouge Parish,12,1703,2072,771,734,991,985,1071,...,771,730,712,788,921,877,905,824,937,West Baton Rouge Parish
743,Louisiana,West Carroll Parish,12,929,1175,360,275,381,297,332,...,333,320,309,325,327,366,395,361,364,West Carroll Parish
755,Louisiana,West Feliciana Parish,12,1238,1168,357,250,630,267,963,...,322,1081,314,914,344,785,374,569,382,West Feliciana Parish


In [61]:
rural_table

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description,Population,is_rural
0,1001,AL,Autauga County,"54,571",2,"Metro - Counties in metro areas of 250,000 to ...",54571,0
1,1003,AL,Baldwin County,"182,265",3,Metro - Counties in metro areas of fewer than ...,182265,1
2,1005,AL,Barbour County,"27,457",6,"Nonmetro - Urban population of 2,500 to 19,999...",27457,1
3,1007,AL,Bibb County,"22,915",1,Metro - Counties in metro areas of 1 million p...,22915,0
4,1009,AL,Blount County,"57,322",1,Metro - Counties in metro areas of 1 million p...,57322,0
...,...,...,...,...,...,...,...,...
3216,72145,PR,Vega Baja Municipio,"59,662",1,Metro - Counties in metro areas of 1 million p...,59662,0
3217,72147,PR,Vieques Municipio,"9,301",7,"Nonmetro - Urban population of 2,500 to 19,999...",9301,1
3218,72149,PR,Villalba Municipio,"26,073",2,"Metro - Counties in metro areas of 250,000 to ...",26073,0
3219,72151,PR,Yabucoa Municipio,"37,941",1,Metro - Counties in metro areas of 1 million p...,37941,0


In [62]:
# pd.merge(data, rural_population, on = 'name')
# rural_population
data = pd.merge(data, rural_table, on="County_Name")
data

,STNAME,CTYNAME,YEAR,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE2024_MALE,AGE2024_FEM,AGE2529_MALE,AGE2529_FEM,AGE3034_MALE,...,AGE6064_MALE,AGE6064_FEM,County_Name,FIPS,State,Population_2010,RUCC_2013,Description,Population,is_rural
0,Georgia,Appling County,12,1478,1779,611,491,620,507,491,...,585,665,Appling County,13001,GA,"18,236",7,"Nonmetro - Urban population of 2,500 to 19,999...",18236,1
1,Georgia,Atkinson County,12,573,610,272,273,280,267,269,...,240,265,Atkinson County,13003,GA,"8,375",9,"Nonmetro - Completely rural or less than 2,500...",8375,1
2,Georgia,Bacon County,12,815,1063,357,321,348,336,346,...,305,368,Bacon County,13005,GA,"11,096",7,"Nonmetro - Urban population of 2,500 to 19,999...",11096,1
3,Georgia,Baker County,12,338,381,78,89,93,76,80,...,128,118,Baker County,12003,FL,"27,115",1,Metro - Counties in metro areas of 1 million p...,27115,0
4,Georgia,Baker County,12,338,381,78,89,93,76,80,...,128,118,Baker County,13007,GA,"3,451",3,Metro - Counties in metro areas of fewer than ...,3451,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14523,Louisiana,Webster Parish,12,3334,4426,1123,1034,1254,1165,1131,...,1264,1331,Webster Parish,22119,LA,"41,207",2,"Metro - Counties in metro areas of 250,000 to ...",41207,0
14524,Louisiana,West Baton Rouge Parish,12,1703,2072,771,734,991,985,1071,...,824,937,West Baton Rouge Parish,22121,LA,"23,788",2,"Metro - Counties in metro areas of 250,000 to ...",23788,0
14525,Louisiana,West Carroll Parish,12,929,1175,360,275,381,297,332,...,361,364,West Carroll Parish,22123,LA,"11,604",9,"Nonmetro - Completely rural or less than 2,500...",11604,1
14526,Louisiana,West Feliciana Parish,12,1238,1168,357,250,630,267,963,...,569,382,West Feliciana Parish,22125,LA,"15,625",2,"Metro - Counties in metro areas of 250,000 to ...",15625,0


In [67]:
data = data.groupby("is_rural").sum()

In [68]:
data

,YEAR,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE2024_MALE,AGE2024_FEM,AGE2529_MALE,AGE2529_FEM,AGE3034_MALE,AGE3034_FEM,AGE3539_MALE,...,AGE4549_FEM,AGE5054_MALE,AGE5054_FEM,AGE5559_MALE,AGE5559_FEM,AGE6064_MALE,AGE6064_FEM,FIPS,RUCC_2013,Population
is_rural,,,,,,,,,,,,,,,,,,,,,
0,43944,37300720,46784528,17291369,16656317,18969966,18298038,17974918,17669064,17332066,...,16668020,16217286,16728836,16981140,17933416,15586912,16946499,109890750,5317,910822700
1,130392,61890583,76410474,26763169,25469523,28448355,27236669,26633749,26068569,25947186,...,25110218,24826600,25411810,26632572,28070095,25103010,27150086,315161402,66762,344399706


In [69]:
data.columns

Index(['YEAR', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM', 'AGE2024_MALE',
       'AGE2024_FEM', 'AGE2529_MALE', 'AGE2529_FEM', 'AGE3034_MALE',
       'AGE3034_FEM', 'AGE3539_MALE', 'AGE3539_FEM', 'AGE4044_MALE',
       'AGE4044_FEM', 'AGE4549_MALE', 'AGE4549_FEM', 'AGE5054_MALE',
       'AGE5054_FEM', 'AGE5559_MALE', 'AGE5559_FEM', 'AGE6064_MALE',
       'AGE6064_FEM', 'FIPS', 'RUCC_2013', 'Population'],
      dtype='object')

In [ ]:
data[">=65, Male"] = data["AGE65PLUS_MALE"]
data[">=65, Female"] = data["AGE65PLUS_FEM"]
data[">=20 <30, Female"] = data["AGE2024_FEM"] + data["AGE2529_FEM"]
data[">=20 <30, Male"] = data["AGE2024_MALE"] + data["AGE2529_MALE"]
data[">=30 <50, Female"] = data["AGE3034_FEM"] + data["AGE3539_FEM"] + data["AGE4044_FEM"] + data["AGE4549_FEM"]
data[">=30 <50, Male"] = data["AGE3034_MALE"] + data["AGE3539_MALE"] + data["AGE4044_MALE"] + data["AGE4549_MALE"]
data[">=50 <65, Female"] = data["AGE5054_MALE"] + data["AGE5559_MALE"] + data["AGE6064_MALE"]
data[">=50 <65, Male"] = data["AGE4044_MALE"] + data["AGE4549_MALE"] + data["AGE5054_MALE"]